In [12]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'charts', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [13]:
#define variables for the process
#
# p / i / s / j / h
ANALYSIS_TYPE = 'p'
PATH_ORIGMP4 = ''
PATH_ROBMP4 = ''

PATH_MODEL = ''
if ANALYSIS_TYPE == 'p':
    PATH_MODEL = ENV_FOLDER_DATA_PROC + 'charts\\' + 'pitchVidNetV3.pt'


In [14]:

import torch
import torch.nn.functional as F



class JWVideoConvNet(torch.nn.Module):
    def __init__(self, in_channels=1, num_classes=41):
        super().__init__()
        # TODO: Initialize network layers
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv3d(in_channels=in_channels, out_channels=4, kernel_size=15, padding=3, stride=5),
            torch.nn.BatchNorm3d(4),
            torch.nn.Sigmoid(),
            torch.nn.MaxPool3d(kernel_size=2, stride=2)
            )
        #self.layer2 = torch.nn.Sequential(
        #    torch.nn.Conv3d(in_channels=4, out_channels=8, kernel_size=8, padding=1, stride=2),
            #torch.nn.BatchNorm3d(16),
        #    torch.nn.Sigmoid(),
            #torch.nn.MaxPool3d(kernel_size=2, stride=2)
        #    )
        #self.layer3 = torch.nn.Sequential(
        #    torch.nn.Conv3d(in_channels=8, out_channels=16, kernel_size=3, padding=1, stride=1),
            #torch.nn.BatchNorm3d(32),
        #    torch.nn.Sigmoid(),
            #torch.nn.MaxPool3d(kernel_size=2, stride=2)
        #    )
        # self.layer4 = nn.Sequential(
        #     nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        #     nn.BatchNorm2d(128),
        #     nn.LeakyReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = torch.nn.Sequential(
            torch.nn.Linear(26460, 4096),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(0.1))
        self.fc2 = torch.nn.Sequential(
            torch.nn.Linear(4096, 1024),
            torch.nn.Sigmoid(),
            torch.nn.Dropout(0.1))
        self.fc3 = torch.nn.Sequential(
            torch.nn.Linear(1024, num_classes))

    def forward(self, x):
        # TODO: Implement the forward pass with using the layers defined above
        #       and the proper activation functions
        #print("input: ", x.shape)
        x = self.layer1(x)
        #print("layer1: ", x.shape)
        #x = self.layer2(x)
        #print("layer2: ", x.shape) 
        #x = self.layer3(x)
        #print("layer3: ", x.shape)
        #x = self.layer4(x)
        #print("layer4: ", x.shape)
        x = torch.flatten(x,1)
        #print("flatten: ", x.shape)
        x = self.fc1(x)
        #print("fc1: ", x.shape)
        x = self.fc2(x)
        #print("fc2: ", x.shape)
        x = self.fc3(x)
        #print("fc3: ", x.shape)
        return x




In [15]:
#debugging

## Model Training 1 ##
import pandas as pd

import pickle
import torchvision
import torch
import glob
import numpy as np
import copy


#get number after # but before space afterwards
folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
folderpath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\'

## get overlap between sourcedf files and targetdf files

#note - source is actually a dict of dict of tensors
dict_sourcedf = {}
list_sourcedf_eps = []
for path_pickle in glob.glob(folderpath_sourcedf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_sourcedf_eps.append(int_epnum)
    dict_sourcedf[int_epnum] = dict_temp

#note - target is actually a df
dict_targetdf = {}
list_targetdf_eps = []
for path_pickle in glob.glob(folderpath_targetdf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_targetdf_eps.append(int_epnum)
    dict_targetdf[int_epnum] = dict_temp

#populate matches
list_eps = []
for src_ep in list_sourcedf_eps:
    if src_ep in list_targetdf_eps:
        list_eps.append(src_ep)

#40 data elements
list_datacols = []
#list_datacols.append('m')
for i in range(40):
    list_datacols.append('d' + str(i))

#define model pieces
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pitch_model = JWVideoConvNet()
pitch_model.load_state_dict(torch.load(ENV_FOLDER_DATA_PROC + 'charts\\' + 'JWPitchModv03.pt'))
pitch_model.to(device=device)

for int_eps in list_eps:
    
    dict_target = {}
    with open(dict_targetdf[int_eps]['path'], 'rb') as file:
        dict_target =  pickle.load(file)
                
    #load video frames into memory
    dict_source = {}
    with open(dict_sourcedf[int_eps]['path'], 'rb') as file:
        dict_source = pickle.load(file)

    

    #loop for data elements
    for key_int,val_frames in dict_source.items():
        int_epnum = int_eps
        int_segnum = key_int
        print('Ep:' + str(int_epnum) + ' Seg:' + str(int_segnum))
        #feed values into the predict / loss / optimize cycle
        #if int_segnum not in dict_target.keys():
        #    continue
        inputs = None
        labels = None

        inputs = []
        #pad inputs
        for x in range(40 - len(inputs)):
            inputs.append( torch.empty((1,360,640), dtype=torch.int8) )
        inputs = torch.stack(inputs[:40], 0)
        inputs_trans = copy.deepcopy(inputs)
        #transform inputs
        
        inputs = inputs.to(device).to(torch.float).movedim(0,1).unsqueeze(0)

        #print(inputs)
        torch.manual_seed(60)
        with torch.no_grad():
            #need to be transformed for JW model
            #not for Jacob's
            model_predictions = pitch_model(inputs)
            print(inputs)
            #print(model_predictions)
        print(model_predictions)
        prediction_data_pts = np.array((model_predictions.to('cpu').detach())[0][1:].tolist()) * 600
        prediction_data_pts[prediction_data_pts > 280] = np.NaN
        prediction_data_pts[prediction_data_pts < 60] = np.NaN
        #print(prediction_data_pts)
        idx = np.isfinite(prediction_data_pts)
        pred_mp, pred_bp = np.polyfit(np.array(list(range(40)))[idx], prediction_data_pts[idx], 1)
        print(pred_mp)

Ep:550 Seg:22
tensor([[[[[  0.,   1., -60.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           ...,
           [ 15.,  15.,  15.,  ...,   0.,   0.,   0.],
           [ 15.,  15.,  15.,  ...,   0.,   0.,   0.],
           [ 15.,  15.,  15.,  ...,   1.,   1.,   1.]],

          [[  0.,   1., -56.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           ...,
           [ 15.,  15.,  15.,  ...,   0.,   0.,   0.],
           [ 15.,  15.,  15.,  ...,   0.,   0.,   0.],
           [ 15.,  15.,  15.,  ...,   1.,   1.,   1.]],

          [[  0.,   1., -52.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
           ...,
           [ 15.,  15.,  15.,  ...,   0.,   0.,   0.],
           [ 15.,  15.,  15.,  ...,   0.,   0.,   0.],

KeyboardInterrupt: 